In [3]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.metrics import make_scorer, log_loss
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

RANDOM_SEED = 777
np.random.seed(RANDOM_SEED)
np.random.SeedSequence(RANDOM_SEED)
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)

In [4]:
metadata = pd.read_csv('../input/is-this-crazy/metadata.csv')
sup_metadata = pd.read_csv('../input/is-this-crazy/supplemental_metadata.csv')
train_labels = pd.read_csv('../input/is-this-crazy/train_labels.csv', index_col="sample_id")
val_labels = pd.read_csv('../input/is-this-crazy/val_labels.csv',index_col='sample_id')
# targets = pd.read_csv('targets.csv',index_col='sample_id')
submission_format = pd.read_csv('../input/is-this-crazy/submission_format.csv')

In [5]:
targets = pd.concat([train_labels,val_labels],axis=0)
targets.shape

(1059, 10)

In [6]:
sam_id = metadata[metadata.instrument_type == 'sam_testbed'].index

In [7]:
# only change the data sets here...
train_path = '../input/ms-gen-train-test-data/train_temp_200.csv'
test_path =  '../input/ms-gen-train-test-data/test_temp_200.csv'
val_path = '../input/ms-gen-train-test-data/val_temp_200.csv'

In [8]:
oof_train_path = 'oof_train_temp_200.csv'
oof_test_path = 'oof_test_temp_200.csv'

In [9]:
%%time
test_100 = pd.read_csv(test_path,index_col=0,header=[0,1])
train_100 = pd.read_csv(train_path,index_col=0,header=[0,1])
val_100 = pd.read_csv(val_path,index_col=0,header=[0,1])

val_100.index = metadata[metadata.split == 'val'].sample_id

val_100.sort_index(inplace=True,axis=1)
test_100.sort_index(inplace=True,axis=1)
train_100.sort_index(inplace=True,axis=1)

CPU times: user 539 ms, sys: 84.1 ms, total: 624 ms
Wall time: 893 ms


In [10]:
# train_100.drop(columns=[('sample_id_encoded', 'Unnamed: 1601_level_1')],axis=1,inplace=True)

In [11]:
train_set_100 = pd.concat([train_100,val_100],axis=0)
test_set_100 = pd.concat([val_100,test_100],axis=0)
train_set_100.shape, test_set_100.shape

((1059, 1000), (804, 1000))

In [12]:
skf = StratifiedKFold(n_splits=20,random_state=RANDOM_SEED,shuffle=True)
log_loss_scorer = make_scorer(log_loss,needs_proba=True)

In [13]:
phy = 'phyllosilicate'
bas = 'basalt'
lr = LogisticRegression(solver='liblinear',C=10,random_state=RANDOM_SEED,penalty='l1')
lr

LogisticRegression(C=10, penalty='l1', random_state=777, solver='liblinear')

In [14]:
def get_coefs(X,y,c):
    lr = LogisticRegression(solver='liblinear',C=c,random_state=RANDOM_SEED,penalty='l1')
    print(cross_val_score(lr,X,y,cv=skf,scoring=log_loss_scorer).mean())
    lr.fit(X,y)
    # print(log_loss(y,lr.predict_proba(X)))
    coefs =  lr.coef_
    coefs_list  = []
    for id,wt in enumerate(tqdm(coefs[0])):
        if wt != 0:
            coefs_list.append(id)
    print(len(coefs_list))
    return coefs_list

In [15]:
def get_mz(mz,df):
    ''' returns list of columns for given mz'''
    cols_list = []
    for col in df.columns:
        if col[0] == str(mz):
            cols_list.append(col)
    return cols_list

In [16]:
def get_total_abundance(df):
    ''' returns total abundance for each sample'''
    tot_abund_df = pd.DataFrame(index=df.index)
    tot_abund_df['tot_abund'] = df.sum(axis=1)
    for mz in tqdm(range(0,100)):
        tot_abund_df[f"tot_abund_{mz}"] = df[get_mz(mz,df)].sum(axis=1)
    print(tot_abund_df.shape)
    return tot_abund_df

In [17]:
def get_grads(df,bin_size=5):
    ''' returns gradients for each mz'''
    grad_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        # df_mz_col1 = df_mz.iloc[:,0].values
        df_mz = df_mz.diff(axis=1)
        # df_mz.iloc[:,0] = df_mz_col1
        df_mz.dropna(axis='columns',inplace=True)
        # df_mz = df_mz/bin_size
        df_mz.columns = [col[0]+ '_' + col[1] + '_' + 'grad' for col in df_mz.columns]
        grad_df = pd.concat([grad_df,df_mz],axis=1)
    print(grad_df.shape)

    return  grad_df

In [18]:
def get_mean_std(df):
    ''' returns mean and std for each mz'''
    mean_std_df = pd.DataFrame(index=df.index)
    for mz in tqdm(range(0,100)):
        df_mz = df.loc[:,get_mz(mz,df)]
        df_mz[f'{mz}_mean'] = df_mz.mean(axis=1)
        df_mz[f'{mz}_std'] = df_mz.std(axis=1)
        mean_std_df = pd.concat([mean_std_df,df_mz],axis=1)
    print(mean_std_df.shape)
    return mean_std_df

In [19]:
# def add_fts(df):
#     tot_abund_df = get_total_abundance(df)
#     grad_df = get_grads(df)
#     mean_std_df = get_mean_std(df)
#     new_df = pd.concat([tot_abund_df,grad_df,mean_std_df],axis=1)
#     return new_df

In [20]:
train_set_100.shape, test_set_100.shape

((1059, 1000), (804, 1000))

In [21]:
assert (train_set_100.columns == test_set_100.columns).all()

In [22]:
t_100 = get_total_abundance(train_set_100)
g_100 = get_grads(train_set_100)
m_100 = get_mean_std(train_set_100)

100%|██████████| 100/100 [00:00<00:00, 233.26it/s]


(1059, 101)


100%|██████████| 100/100 [00:01<00:00, 92.41it/s]


(1059, 900)


100%|██████████| 100/100 [00:00<00:00, 101.56it/s]

(1059, 1200)


In [23]:
t_100_test = get_total_abundance(test_set_100)
g_100_test = get_grads(test_set_100)
m_100_test = get_mean_std(test_set_100)


100%|██████████| 100/100 [00:00<00:00, 240.05it/s]


(804, 101)


100%|██████████| 100/100 [00:01<00:00, 99.55it/s]


(804, 900)


100%|██████████| 100/100 [00:00<00:00, 107.10it/s]

(804, 1200)


In [24]:
t_100_test.sort_index(inplace=True,axis=1)
t_100.sort_index(inplace=True,axis=1)

m_100_test.sort_index(inplace=True,axis=1)
m_100.sort_index(inplace=True,axis=1)

g_100_test.sort_index(inplace=True,axis=1)
g_100.sort_index(inplace=True,axis=1)

In [25]:
assert (t_100.columns == t_100_test.columns).all()
assert (g_100.columns == g_100_test.columns).all()
assert (m_100.columns == m_100_test.columns).all()

In [26]:
ntrain = 1059
ntest = 804
NFOLDS = 20

In [27]:
oof_train_df = pd.DataFrame(index=metadata[metadata.split != 'test'].index, columns=['sample_id',*targets.columns])
oof_test_df = pd.DataFrame(index=metadata[metadata.split != 'train'].index, columns=['sample_id',*targets.columns])

In [28]:
agg_loss = []
for target in targets.columns:
    print('TARGET ',target)
    imp_fts = {}
    
    #  get all imp coefs for the target
    t_100_coefs = get_coefs(t_100,targets[target],5)
    g_100_coefs = get_coefs(g_100,targets[target],5)
    m_100_coefs = get_coefs(m_100,targets[target],10)
    o_100_coefs = get_coefs(train_set_100,targets[target],10)
    
    imp_fts['o_100_coefs'] = o_100_coefs
    imp_fts['t_100_coefs'] = t_100_coefs
    imp_fts['g_100_coefs'] = g_100_coefs
    imp_fts['m_100_coefs'] = m_100_coefs

    # get all imp_dfs and concat them
    train_imp_ = train_set_100.iloc[:,o_100_coefs]
    t_100_imp_ = t_100.iloc[:,t_100_coefs]
    g_100_imp_ = g_100.iloc[:,g_100_coefs]
    m_100_imp_ = m_100.iloc[:,m_100_coefs]

    print(train_imp_.shape,t_100_imp_.shape,g_100_imp_.shape,m_100_imp_.shape)
    X = pd.concat([train_imp_,t_100_imp_,g_100_imp_,m_100_imp_],axis=1)
    print(X.shape)

    # same goes for test set
    test_imp_ = test_set_100.iloc[:,o_100_coefs]
    t_100_imp_test = t_100_test.iloc[:,t_100_coefs]
    g_100_imp_test = g_100_test.iloc[:,g_100_coefs]
    m_100_imp_test = m_100_test.iloc[:,m_100_coefs]

    print(test_imp_.shape,t_100_imp_test.shape,g_100_imp_test.shape,m_100_imp_test.shape)
    
    X_test = pd.concat([test_imp_, t_100_imp_test, g_100_imp_test, m_100_imp_test],axis=1)
    print(X_test.shape)
    
    #  add feature
    X['is_sam'] = 0
    X.loc[sam_id[:12],'is_sam'] = 1
    X_test['is_sam'] = 0
    X_test.loc[sam_id[12:],'is_sam'] = 1
    
    
    assert (X.columns == X_test.columns).all()

    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    Y = targets[target]

    cb_clf = CatBoostClassifier(verbose=999,random_seed=RANDOM_SEED,n_estimators=5000)
    cv_loss = []
    for fold, (train_id, test_id) in enumerate(tqdm(skf.split(X,Y))):

        X_train, Y_train = X.iloc[train_id,], Y.iloc[train_id,]
        X_val, Y_val = X.iloc[test_id,], Y.iloc[test_id,]
        #  ohh man unwanted semi colon in iloc throws an error...
        
        # train model  and pred on oof folds
        cb_clf.fit(X_train.values,Y_train)
        y_preds = cb_clf.predict_proba(X_val.values)[:,1]
        fold_loss = log_loss(Y_val,y_preds)
        print(f'fold Loss {fold}: ',fold_loss)
        cv_loss.append(fold_loss)

        oof_train[test_id] = y_preds
        oof_test_skf[fold,:] = cb_clf.predict_proba(X_test.values)[:,1]
        
        print("---------------------------------------------------------------------------")
        
    print('Training Loss:', log_loss(Y,cb_clf.predict_proba(X.values)[:,1]))
    
    oof_test[:] = oof_test_skf.mean(axis=0)
    oof_test_df[target] = oof_test
    oof_train_df[target] = oof_train

    print(f'Loss for {target}:')
    print(np.array(cv_loss).mean(),np.array(cv_loss).std())
    agg_loss.append(np.array(cv_loss).mean())
    print("==================================================================================================")

print('AGG Loss:',np.array(agg_loss))
print('AGG Loss mean:',np.array(agg_loss).mean())

TARGET  basalt
0.2968182236100406


100%|██████████| 101/101 [00:00<00:00, 426610.98it/s]

22


0.2983079227821478


100%|██████████| 900/900 [00:00<00:00, 638186.58it/s]

...

4995:	learn: 0.0031453	total: 1m 29s	remaining: 71.4ms
4999:	learn: 0.0031420	total: 1m 29s	remaining: 0us
fold Loss 17:  0.08365596815208529
---------------------------------------------------------------------------
Learning rate set to 0.002361
0:	learn: 0.6889719	total: 35.3ms	remaining: 2m 56s
999:	learn: 0.0354525	total: 18.2s	remaining: 1m 12s
1998:	learn: 0.0139289	total: 36s	remaining: 54.1s
2997:	learn: 0.0076476	total: 54.4s	remaining: 36.3s
3996:	learn: 0.0047599	total: 1m 12s	remaining: 18.2s


19it [28:51, 91.11s/it]

4995:	learn: 0.0032575	total: 1m 30s	remaining: 72.5ms
4999:	learn: 0.0032544	total: 1m 30s	remaining: 0us
fold Loss 18:  0.06639209140374909
---------------------------------------------------------------------------
Learning rate set to 0.002362
0:	learn: 0.6890196	total: 28.2ms	remaining: 2m 20s
999:	learn: 0.0338097	total: 18.1s	remaining: 1m 12s
1998:	learn: 0.0133584	total: 36.1s	remaining: 54.1s
2997:	learn: 0.0074051	total: 54.3s	remaining: 36.2s
3996:	learn: 0.0044456	total: 1m 12s	remaining: 18.2s


20it [30:23, 91.17s/it]

4995:	learn: 0.0031235	total: 1m 30s	remaining: 72.5ms
4999:	learn: 0.0031192	total: 1m 30s	remaining: 0us
fold Loss 19:  0.10877240098128994
---------------------------------------------------------------------------
Training Loss: 0.00831134472287677
Loss for sulfide:
0.06585137909859463 0.05502963226134115
AGG Loss: [0.13200745 0.08674289 0.14688047 0.19067936 0.00983821 0.13258772
 0.19645603 0.17209051 0.16651203 0.06585138]
AGG Loss mean: 0.12996460439771276


In [29]:
print(np.array(agg_loss).mean())

0.12996460439771276


In [30]:
test_preds = oof_test_df.iloc[:293]
train_preds = oof_train_df.iloc[766:]

In [31]:
display(test_preds)
display(train_preds)

,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.002304,0.001244,0.017878,0.009066,0.000119,0.005200,0.006248,0.069620,0.003726,0.000603
767,NaN,0.024833,0.005837,0.002922,0.011116,0.000051,0.005315,0.008344,0.107110,0.017575,0.000807
768,NaN,0.824128,0.072721,0.001528,0.056458,0.000546,0.009065,0.863309,0.867325,0.043452,0.000929
769,NaN,0.009914,0.004090,0.040709,0.018084,0.000046,0.963984,0.019731,0.004481,0.946220,0.000744
770,NaN,0.004268,0.002670,0.008930,0.806692,0.000062,0.990266,0.944705,0.003653,0.007387,0.000197
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.004305,0.001442,0.034767,0.012648,0.000084,0.004353,0.006034,0.028033,0.006299,0.000281
1055,NaN,0.008403,0.004920,0.006829,0.036553,0.000707,0.042862,0.007590,0.848290,0.002577,0.000549
1056,NaN,0.004083,0.007802,0.001318,0.013079,0.998677,0.002993,0.005814,0.003015,0.002937,0.000366
1057,NaN,0.005260,0.929231,0.008808,0.029624,0.000643,0.005740,0.036662,0.009146,0.004461,0.000956


,sample_id,basalt,carbonate,chloride,iron_oxide,oxalate,oxychlorine,phyllosilicate,silicate,sulfate,sulfide
766,NaN,0.002534,0.001378,0.029344,0.011450,0.000106,0.005783,0.007538,0.282085,0.003611,0.000761
767,NaN,0.043733,0.002228,0.003144,0.015892,0.000051,0.005746,0.013382,0.659950,0.023246,0.000830
768,NaN,0.232021,0.513448,0.002042,0.114108,0.000454,0.008556,0.595695,0.527308,0.179847,0.000745
769,NaN,0.019129,0.004954,0.092810,0.026938,0.000053,0.929979,0.025062,0.005891,0.839138,0.000722
770,NaN,0.003224,0.003263,0.011432,0.315233,0.000064,0.989705,0.898091,0.004493,0.009680,0.000208
...,...,...,...,...,...,...,...,...,...,...,...
1054,NaN,0.006688,0.001489,0.039393,0.017425,0.000111,0.004877,0.007823,0.074827,0.007771,0.000199
1055,NaN,0.012000,0.006462,0.007690,0.079234,0.000654,0.130243,0.007728,0.269195,0.002543,0.000627
1056,NaN,0.003742,0.010521,0.001332,0.014964,0.998562,0.003045,0.007501,0.003521,0.003248,0.000384
1057,NaN,0.010419,0.813453,0.013334,0.038222,0.000228,0.010152,0.063361,0.009599,0.005775,0.000769


In [32]:
agg_loss_train = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],train_preds[tar])
    print(tar, loss)
    agg_loss_train.append(loss)
print(np.array(agg_loss_train).mean())

basalt 0.1402455406987601
carbonate 0.08295032961081271
chloride 0.16515437520611542
iron_oxide 0.23095890769728036
oxalate 0.0023015624229882395
oxychlorine 0.14718917356319441
phyllosilicate 0.1759032369287015
silicate 0.14968317398064931
sulfate 0.19034961542507395
sulfide 0.06232238250556354
0.13470582980391393


In [33]:
agg_loss_test = []
for tar in targets.columns:
    loss = log_loss(val_labels[tar],test_preds[tar])
    print(tar, loss)
    agg_loss_test.append(loss)
print(np.array(agg_loss_test).mean())

basalt 0.022449940121216937
carbonate 0.014067263256509267
chloride 0.029188302727019156
iron_oxide 0.05373334915612052
oxalate 0.0005452257422163027
oxychlorine 0.026236428911909967
phyllosilicate 0.04574923853000603
silicate 0.031061487849986594
sulfate 0.029855270532075165
sulfide 0.0045413245699367
0.025742783139699667


In [34]:
oof_test_df.sample_id = metadata[metadata.split != 'train'].sample_id
oof_train_df.sample_id = metadata[metadata.split != 'test'].sample_id

In [35]:
oof_train_df.to_csv(oof_train_path, index=False)
oof_test_df.to_csv(oof_test_path, index=False)